# Задание
Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события

- event - событие

app_install – установка приложения
app_start – открыть приложения
registration – зарегистрироваться 
search – перейти на страницу поиска товаров (каталог)
open_item – открыть товар
choose_item – отправить товар в корзину
tap_basket – перейти в корзину
purchase – подтверждение покупки
- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь

yandex-direct – Яндекс директ
google_ads – реклама в Google
vk_ads – реклама в ВК
instagram_ads – реклама в instagram
facebook_ads – реклама в facebook
referral – акция «приведи друга»

In [207]:
import pandas as pd
import seaborn as sns

In [3]:
case = pd.read_csv("D://KARPOV/KC_case_data.csv", encoding='utf-8', parse_dates=['date'])


In [111]:
case.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


# 1.
Найдите MAU февраля

In [5]:

case.groupby(pd.Grouper(key='date', freq='1M')).agg({'device_id':'nunique'})

,device_id
date,
2020-01-31,99161
2020-02-29,75032
2020-03-31,74623


#### MAU февраля - 75032

# 2. 
Найдите количество установок в январе

In [154]:
case.event.value_counts()


app_start      748705
search         708639
choose_item    538669
tap_basket     377665
app_install    154597
purchase       141383
register        78310
Name: event, dtype: int64

In [6]:
#Сделаем сводную таблицу 
case.groupby(['event',pd.Grouper(key='date', freq='1M')]).agg(count=('event','count')).reset_index().pivot(index='date', columns='event', values='count')

event,app_install,app_start,choose_item,purchase,register,search,tap_basket
date,,,,,,,
2020-01-31,80297,268471,196137,59208,42946,254732,140616
2020-02-29,38078,243694,174911,44594,20578,230689,122181
2020-03-31,36222,236540,167621,37581,14786,223218,114868


#### Видим, что количество установок в январе = 80297

# 3. 
Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [7]:
case.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [8]:
#case['one_week']= case.query('event == "app_install"')['date'] - case.query('event == "purchase"')['date']
case_new=case.query('event == "app_install" or event == "purchase"').groupby(['device_id', 'date','event']).agg(count=('event','count')).sort_values('date')

In [9]:
case_new.head(3)

,,,count
device_id,date,event,
936996,2020-01-01,purchase,1
32817948,2020-01-01,app_install,1
25746914,2020-01-01,app_install,1


In [10]:

case_new=case_new.reset_index().groupby(['device_id', 'event']).agg(date_action=('date','first')).sort_values('device_id').reset_index()


In [11]:
case_new=case_new.pivot(index='device_id', columns='event', values='date_action')
case_new.head(15)

event,app_install,purchase
device_id,,
4013,2020-01-15,NaT
4014,NaT,2020-02-01
4016,2020-01-04,NaT
4046,2020-01-04,2020-01-04
4060,NaT,2020-03-07
4065,2020-03-19,NaT
4067,2020-01-31,NaT
4076,2020-01-10,NaT
4088,2020-01-16,NaT


Получаем таблицу, где видим, что, например, device_id = 4046 зарегистрировался и в тот же день совершил первую покупку.
При этом есть пользователи, которые совершили покупку, но не установили приложение. 
Это объясняется тем, что у них уже оно было уствновлено.

In [12]:
case_new['days_diff']=case_new['purchase']-case_new['app_install']

case_new
#merged = merged[merged['difference'].dt.days <= 7]

event,app_install,purchase,days_diff
device_id,,,
4013,2020-01-15,NaT,NaT
4014,NaT,2020-02-01,NaT
4016,2020-01-04,NaT,NaT
4046,2020-01-04,2020-01-04,0 days
4060,NaT,2020-03-07,NaT
...,...,...,...
35379281,2020-03-29,NaT,NaT
35380796,2020-03-31,NaT,NaT
35381595,2020-03-30,NaT,NaT


In [13]:
case_new['seven_days'] =case_new['days_diff'] <= '7 days'
case_new

event,app_install,purchase,days_diff,seven_days
device_id,,,,
4013,2020-01-15,NaT,NaT,False
4014,NaT,2020-02-01,NaT,False
4016,2020-01-04,NaT,NaT,False
4046,2020-01-04,2020-01-04,0 days,True
4060,NaT,2020-03-07,NaT,False
...,...,...,...,...
35379281,2020-03-29,NaT,NaT,False
35380796,2020-03-31,NaT,NaT,False
35381595,2020-03-30,NaT,NaT,False


In [14]:
case_new = case_new.groupby('app_install').seven_days.agg({'count', 'sum'})
case_new

,sum,count
app_install,,
2020-01-01,1408,3579
2020-01-02,1186,3144
2020-01-03,834,2402
2020-01-04,639,1831
2020-01-05,587,1671
...,...,...
2020-03-27,277,1199
2020-03-28,209,1091
2020-03-29,223,1117


In [15]:
case_new['cr']= (case_new['sum'] / case_new['count']).mul(100).round(1)
case_new.sort_values('cr', ascending=False)

,sum,count,cr
app_install,,,
2020-01-01,1408,3579,39.3
2020-01-09,558,1424,39.2
2020-01-15,1650,4310,38.3
2020-01-14,1973,5173,38.1
2020-01-02,1186,3144,37.7
...,...,...,...
2020-03-18,235,1171,20.1
2020-03-29,223,1117,20.0
2020-03-22,251,1261,19.9


#### Наибольшая конверсия была для когорты 2020-01-01 и составила 39,1%

# 4.
С какого платного маркетингового канала пришло больше всего новых пользователей? 

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

NB! В этом и подобном ему заданиях писать надо только название, без кавычек.



In [16]:
case.groupby('utm_source').agg({'device_id':'nunique'}).sort_values('device_id', ascending=False)

,device_id
utm_source,
-,52273
yandex-direct,40712
google_ads,38096
vk_ads,34086
instagram_ads,31048
facebook_ads,25959
referal,15926


#### Больше всего новых пользователей пришло к нам из Яндекс.Директ

# 5.
Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: 
- «Поиск», 
- «Добавление товара»,  
- «Переход в корзину», 
- «Регистрация».

Подсказка: 

- Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации. Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 



In [156]:
funnel1=case.iloc[:, [0, 1, 4, 7]]
funnel1

,date,event,device_id,utm_source
0,2020-01-01,app_start,669460,-
1,2020-01-01,app_start,833621,vk_ads
2,2020-01-01,app_start,1579237,referal
3,2020-01-01,app_start,1737182,facebook_ads
4,2020-01-01,app_start,4029024,facebook_ads
...,...,...,...,...
2747963,2020-03-31,register,2984778,facebook_ads
2747964,2020-03-31,register,27301864,-
2747965,2020-03-31,register,1294285,google_ads
2747966,2020-03-31,register,3010574,google_ads


In [157]:
funnel1=funnel1.groupby(['device_id', 'date', 'event'], as_index=False).agg(count=('device_id','count'))

In [158]:
funnel1=funnel1.groupby(['device_id', 'event'], as_index=False).agg({'date':'first'})
funnel1

,device_id,event,date
0,4013,app_install,2020-01-15
1,4013,app_start,2020-01-15
2,4013,choose_item,2020-01-15
3,4013,search,2020-01-15
4,4013,tap_basket,2020-01-15
...,...,...,...
960254,35397767,app_install,2020-03-31
960255,35397767,app_start,2020-03-31
960256,35397767,choose_item,2020-03-31
960257,35397767,search,2020-03-31


In [159]:
funnel1=funnel1.pivot(index='device_id', columns='event', values='date')
funnel1.head(5)

event,app_install,app_start,choose_item,purchase,register,search,tap_basket
device_id,,,,,,,
4013,2020-01-15,2020-01-15,2020-01-15,NaT,NaT,2020-01-15,2020-01-15
4014,NaT,2020-01-02,2020-01-02,2020-02-01,2020-02-01,2020-01-02,2020-01-02
4016,2020-01-04,2020-01-04,2020-01-04,NaT,NaT,2020-01-04,2020-02-15
4018,NaT,2020-03-27,2020-03-27,NaT,NaT,2020-03-27,2020-03-27
4046,2020-01-04,2020-01-04,2020-01-04,2020-01-04,2020-01-04,2020-01-04,2020-01-04


In [160]:
funnel1['registration'] = funnel1['purchase']> funnel1['register']


#### Для зарегистрированных пользователей

In [161]:
not_reg=funnel1.query('registration == False')
reg = funnel1.query('registration == True')
reg.head(3)
#already_registered

event,app_install,app_start,choose_item,purchase,register,search,tap_basket,registration
device_id,,,,,,,,
4214,NaT,2020-01-03,2020-01-03,2020-01-28,2020-01-13,2020-01-03,2020-01-04,True
4767,NaT,2020-01-05,2020-01-05,2020-01-15,2020-01-06,2020-01-05,2020-01-06,True
5205,2020-01-04,2020-01-04,2020-01-04,2020-01-09,2020-01-07,2020-01-04,2020-01-04,True


In [23]:
#reg.groupby('device_id').agg({'tap_basket':'count', 'register':'count'}).sum()
reg.groupby('device_id').agg({'search':'count', 'choose_item':'count', 'tap_basket':'count', 'register':'count', 'purchase':'count'}).sum()

search         3122
choose_item    3122
tap_basket     3122
register       3122
purchase       3122
dtype: int64

#### Среди незарегистрированных

In [24]:
convertion=not_reg.groupby('device_id').agg({'app_install':'nunique','app_start':'nunique','search':'nunique', 'choose_item':'nunique', 'tap_basket':'nunique', 'register':'nunique', 'purchase':'nunique'}).sum()

In [25]:
convertion

app_install    152105
app_start      187762
search         181366
choose_item    152569
tap_basket     122292
register        75188
purchase        67753
dtype: int64

In [26]:
print('Конверсия=', round(convertion['register']/convertion['tap_basket'],3)*100, sep='')

Конверсия=61.5


#### Как мы видим, самая низкая коверсия наблюдается из этапа "Переход к корзину" в  этап "Регистрация". Тут отваливается более 60% пользователей

# 6.
Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

Будем считать: 
CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [128]:
source=case.iloc[:, [0, 1, 4, 7]]
source

,date,event,device_id,utm_source
0,2020-01-01,app_start,669460,-
1,2020-01-01,app_start,833621,vk_ads
2,2020-01-01,app_start,1579237,referal
3,2020-01-01,app_start,1737182,facebook_ads
4,2020-01-01,app_start,4029024,facebook_ads
...,...,...,...,...
2747963,2020-03-31,register,2984778,facebook_ads
2747964,2020-03-31,register,27301864,-
2747965,2020-03-31,register,1294285,google_ads
2747966,2020-03-31,register,3010574,google_ads


In [129]:
source=source.groupby(['device_id','date',  'event','utm_source'], as_index=False).agg(count=('device_id','count'))

In [130]:
source=source.query("event == 'app_start' | event == 'purchase'").query('utm_source != "-"').groupby(['utm_source', 'device_id', 'event'], as_index=False).agg(date_action=('date','first'))

In [131]:
source=source.pivot(index=['device_id', 'utm_source'], columns='event', values='date_action').reset_index()
source

event,device_id,utm_source,app_start,purchase
0,4013,vk_ads,2020-01-15,NaT
1,4014,referal,2020-01-02,2020-02-01
2,4016,google_ads,2020-02-18,NaT
3,4016,instagram_ads,2020-01-06,NaT
4,4016,yandex-direct,2020-01-04,NaT
...,...,...,...,...
185822,35379281,vk_ads,2020-03-29,NaT
185823,35380796,referal,2020-03-31,NaT
185824,35381595,facebook_ads,2020-03-31,NaT
185825,35388218,yandex-direct,2020-03-31,NaT


In [132]:
source1 = source.groupby('utm_source')
source1.purchase.count() / source1.app_start.count()

utm_source
facebook_ads     0.347382
google_ads       0.297674
instagram_ads    0.346658
referal          0.399523
vk_ads           0.362804
yandex-direct    0.295477
dtype: float64

In [133]:
(source1.purchase.count() / source1.app_start.count()).idxmin()

'yandex-direct'

#### Получается, что самая низкая конверсия среди тех, кто пришел из "Яндекс.Директ"

# 7.
Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [147]:
new = case.iloc[:, [2, 4]]
new

,purchase_sum,device_id
0,NaN,669460
1,NaN,833621
2,NaN,1579237
3,NaN,1737182
4,NaN,4029024
...,...,...
2747963,NaN,2984778
2747964,NaN,27301864
2747965,NaN,1294285
2747966,NaN,3010574


In [162]:
merged=source.merge(new, on='device_id').drop_duplicates()

In [163]:
merged=merged.dropna().groupby(['utm_source', 'device_id'], as_index=False).agg({'purchase':'first', 'purchase_sum':'sum'})
merged

,utm_source,device_id,purchase,purchase_sum
0,facebook_ads,4046,2020-01-11,2718.0
1,facebook_ads,4211,2020-01-24,606.5
2,facebook_ads,4292,2020-01-10,8796.0
3,facebook_ads,4602,2020-02-13,280.5
4,facebook_ads,4767,2020-01-15,2094.0
...,...,...,...,...
61844,yandex-direct,35245401,2020-03-29,202.5
61845,yandex-direct,35258588,2020-03-30,157.0
61846,yandex-direct,35266881,2020-03-25,735.0
61847,yandex-direct,35277913,2020-03-20,81.0


In [164]:
merged.groupby('utm_source').agg({'purchase_sum':'median'}).sort_values('purchase_sum', ascending=False)

,purchase_sum
utm_source,
referal,595.75
instagram_ads,582.00
facebook_ads,578.50
vk_ads,573.00
google_ads,551.00
yandex-direct,548.00


#### Выходит, что медианная сумма покупки была больше всех у пользователей, пришедших с реферальной программы

# 8.
Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения имеет самый высокий ROMI? 

In [197]:
romi = merged.groupby('utm_source').agg({'purchase_sum':'sum', 'device_id':'nunique'})

In [198]:
romi=romi.reset_index()
romi

,utm_source,purchase_sum,device_id
0,facebook_ads,14486439.0,9015
1,google_ads,16433931.5,11335
2,instagram_ads,17057372.5,10759
3,referal,10208460.0,6360
4,vk_ads,19393029.0,12357
5,yandex-direct,17360192.5,12023


In [199]:
romi['expenditure']=pd.DataFrame(data={'expenditure': [ 8590498, 10534878, 8561626,2*636000,9553531,10491707]})

In [205]:
romi['romi']=(romi['purchase_sum']-romi['expenditure'])/romi['expenditure']

In [206]:
romi.sort_values('romi', ascending=False)

,utm_source,purchase_sum,device_id,expenditure,margin,romi
3,referal,10208460.0,6360,1272000,7.025519,7.025519
4,vk_ads,19393029.0,12357,9553531,1.029933,1.029933
2,instagram_ads,17057372.5,10759,8561626,0.992305,0.992305
0,facebook_ads,14486439.0,9015,8590498,0.686333,0.686333
5,yandex-direct,17360192.5,12023,10491707,0.654659,0.654659
1,google_ads,16433931.5,11335,10534878,0.559955,0.559955


#### Среди платных каналов - VK имеет наибольший показатель ROMI